In [6]:
import os
os.getcwd()

'/home/nusai/.cache/bazel/_bazel_nusai/6c77ff2c63f199c891700691046ca70c/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/isaac_data_parser/parse_data.runfiles/com_nvidia_isaac'

In [7]:
#go back to library path.
os.chdir("..")
os.getcwd()

'/home/nusai/.cache/bazel/_bazel_nusai/6c77ff2c63f199c891700691046ca70c/execroot/com_nvidia_isaac/bazel-out/k8-opt/bin/isaac_data_parser/parse_data.runfiles'

In [13]:
data_path=[]
data_path.append('/home/nusai/spot-project/spot_data_v2/f37e2316-289e-11eb-b222-bf24542cb1c0/17-11-20000')
data_path.append('/home/nusai/spot-project/spot_data_v2/f37e2316-289e-11eb-b222-bf24542cb1c0/17-11-20001')
data_path.append('/home/nusai/spot-project/spot_data_v2/bec9d6cc-28b9-11eb-9ee2-e9e1f1528ba6/17-11-20000')
data_path.append('/home/nusai/spot-project/spot_data_v2/63052784-28c2-11eb-be65-970da4afa99c/17-11-20000')
data_path.append('/home/nusai/spot-project/spot_data_v2/16af3598-28a7-11eb-aafd-8f41411dfcc3/17-11-20000')
data_path.append('/home/nusai/spot-project/spot_data_v2/16af3598-28a7-11eb-aafd-8f41411dfcc3/17-11-20001')
data_path.append('/home/nusai/spot-project/spot_data_v2/f5266a1c-28bb-11eb-83b6-e9f35fbef075/17-11-20000')

try:
    os.mkdir(M_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % M_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % M_EXPORT_DIR)



try:
    os.mkdir(L_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % L_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % L_EXPORT_DIR)



try:
    os.mkdir(R_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % R_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % R_EXPORT_DIR)



from engine.pyalice import Cask, Message
import io
from PIL import Image

import cv2
import csv
import numpy
from turbojpeg import TurboJPEG, TJPF_GRAY, TJSAMP_GRAY, TJFLAG_PROGRESSIVE

jpeg = TurboJPEG()

# frame intention_type current_velocity steering_wheel_angle dlm
f = open(os.path.join(LABEL_DIR, 'label.txt'), 'w')
labelwriter = csv.writer(f, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
labelwriter.writerow(['frame', 'intention_type', 'current_velocity', 'steering_wheel_angle', 'dlm'])
    
    
intentions = {
    0 : "forward",
    1 : "left",
    2 : "right",
    3 : "unknown"
}



for path in data_path:
    CASK_PATH=path
    cask = Cask(CASK_PATH)

    M_EXPORT_DIR="/home/nusai/spot-project/spot_data_v2/parsed/rgb_m/"
    L_EXPORT_DIR="/home/nusai/spot-project/spot_data_v2/parsed/rgb_l/"
    R_EXPORT_DIR="/home/nusai/spot-project/spot_data_v2/parsed/rgb_r/"
    LABEL_DIR = "/home/nusai/spot-project/spot_data_v2/parsed/"


    mid_channel=None
    control_channel=None
    left_channel = None
    right_channel = None

    for channel in cask.channels:
        if channel.name == "mid_color":
            mid_channel = channel
        if channel.name == "left_color":
            left_channel = channel
        if channel.name == "right_color":
            right_channel = channel
        if channel.name == "intention_control":
            control_channel = channel


    # procees idx first
    control_idx = []
    last_pubtime = 0
    last_proto = None

    for idx,msg in enumerate(mid_channel):
        try:
            byteString =  msg.tensor.T[0]
            fObj = open(os.path.join(M_EXPORT_DIR, f"{msg.pubtime}.jpg"), "wb")
            fObj.write(byteString)
            fObj.close()

            for idx_control, msg_control in enumerate(control_channel):
                if len(control_idx) > 2 and idx_control < control_idx[-1]:
                    continue

        #         print(control_idx, idx_control,idx)


                if msg_control.pubtime < msg.pubtime:
                    last_pubtime = msg_control.pubtime
                    last_proto = msg_control.proto
                    continue
                else:
                    diff_this = msg_control.pubtime - msg.pubtime
                    diff_last = abs(last_pubtime - msg.pubtime)

                    if diff_this < diff_last:
                        control_idx.append(idx_control)
                        labelwriter.writerow([msg.pubtime, 'DLM', msg_control.proto.x, msg_control.proto.y, intentions[int(msg_control.proto.z)]])
        #    
                    else:
                        control_idx.append(idx_control-1)
                        labelwriter.writerow([msg.pubtime, 'DLM', last_proto.x, last_proto.y, intentions[int(last_proto.z)]])
        #             print("this: ", msg_control.pubtime, msg.pubtime, )
        #             print("last: ", last_pubtime, msg.pubtime, )
                    break
        except:
            print("Skipped")


    # Left images 
    control_idx = []
    last_pubtime = 0
    last_tensor = None
    for idx,msg in enumerate(mid_channel):
        try:
            for idx_control, msg_control in enumerate(left_channel):
                if len(control_idx) > 2 and idx_control < control_idx[-1]:
                    continue

                if msg_control.pubtime < msg.pubtime:
                    last_pubtime = msg_control.pubtime
                    last_tensor = msg_control.tensor
                    continue
                else:
                    diff_this = msg_control.pubtime - msg.pubtime
                    diff_last = abs(last_pubtime - msg.pubtime)

                    if diff_this < diff_last:
                        control_idx.append(idx_control)
                        byteString =  msg_control.tensor.T[0]
                        fObj = open(os.path.join(L_EXPORT_DIR, f"{msg.pubtime}.jpg"), "wb")
                        fObj.write(byteString)
                        fObj.close()
                    else:
                        control_idx.append(idx_control-1)
                        byteString =  last_tensor.T[0]
                        fObj = open(os.path.join(L_EXPORT_DIR, f"{msg.pubtime}.jpg"), "wb")
                        fObj.write(byteString)
                        fObj.close()
                    break
        except:
            print("skipped")

    # Right Images
    control_idx = []
    last_pubtime = 0
    last_tensor = None
    for idx,msg in enumerate(mid_channel):
    
        try:
            for idx_control, msg_control in enumerate(right_channel):
                if len(control_idx) > 2 and idx_control < control_idx[-1]:
                    continue 

        #         print(control_idx, idx_control,idx)

                if msg_control.pubtime < msg.pubtime:
                    last_pubtime = msg_control.pubtime
                    last_tensor = msg_control.tensor
                    continue
                else:
                    diff_this = msg_control.pubtime - msg.pubtime
                    diff_last = abs(last_pubtime - msg.pubtime)

                    if diff_this < diff_last:
                        control_idx.append(idx_control)
                        byteString =  msg_control.tensor.T[0]
                        fObj = open(os.path.join(R_EXPORT_DIR, f"{msg.pubtime}.jpg"), "wb")
                        fObj.write(byteString)
                        fObj.close()
                    else:
                        control_idx.append(idx_control-1)
                        byteString =  last_tensor.T[0]
                        fObj = open(os.path.join(R_EXPORT_DIR, f"{msg.pubtime}.jpg"), "wb")
                        fObj.write(byteString)
                        fObj.close()
                    break
                
        except:
            print("skipped")

    print("Finished")
print("Done")

Creation of the directory /home/nusai/spot-project/spot_data_v2/parsed/rgb_m/ failed
Creation of the directory /home/nusai/spot-project/spot_data_v2/parsed/rgb_l/ failed
Creation of the directory /home/nusai/spot-project/spot_data_v2/parsed/rgb_r/ failed
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
S

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


...

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


In [ ]:
CASK_PATH='/home/nusai/spot-project/spot/demo000'
M_EXPORT_DIR="/home/nusai/spot-project/spot/data/rgb_0/"
L_EXPORT_DIR="/home/nusai/spot-project/spot/data/rgb_4/"
R_EXPORT_DIR="/home/nusai/spot-project/spot/data/rgb_6/"
LABEL_DIR = "/home/nusai/spot-project/spot/data/"
try:
    os.mkdir(M_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % M_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % M_EXPORT_DIR)



try:
    os.mkdir(L_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % L_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % L_EXPORT_DIR)



try:
    os.mkdir(R_EXPORT_DIR)
except OSError:
    print ("Creation of the directory %s failed" % R_EXPORT_DIR)
else:
    print ("Successfully created the directory %s " % R_EXPORT_DIR)




from engine.pyalice import Cask, Message
import io
from PIL import Image
cask = Cask(CASK_PATH)

import cv2
import csv
import numpy
from turbojpeg import TurboJPEG, TJPF_GRAY, TJSAMP_GRAY, TJFLAG_PROGRESSIVE

jpeg = TurboJPEG()


# frame intention_type current_velocity steering_wheel_angle dlm
f = open(os.path.join(LABEL_DIR, 'label.txt'), 'w')
labelwriter = csv.writer(f, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
labelwriter.writerow(['frame', 'intention_type', 'current_velocity', 'steering_wheel_angle', 'dlm'])
    
    
intentions = {
    0 : "forward",
    1 : "left",
    2 : "right",
    3 : "backward"
}

# List all channels recorded
index = 1
for channel in cask.channels:
    print (channel)
    if channel.name == "mid_color":
        for idx, msg in enumerate(channel):  # Goes through every messages one by one in recorded order
#             print(msg.proto)
            byteString =  msg.tensor.T[0]
            fObj = open(os.path.join(M_EXPORT_DIR, f"{idx}.jpg"), "wb")
            fObj.write(byteString)
            fObj.close()
            
#             print(msg)
#             index = idx
#             if index > 300:
#                 break
#             index = index+1
            
    if channel.name == "left_color":
        for idx, msg in enumerate(channel):  # Goes through every messages one by one in recorded order
#             print(msg.proto)
            byteString =  msg.tensor.T[0]
            fObj = open(os.path.join(L_EXPORT_DIR, f"{idx}.jpg"), "wb")
            fObj.write(byteString)
            fObj.close()
            
#             index = idx
#             if index > 300:
#                 break
#             index = index+1
            
    if channel.name == "right_color":
        for idx, msg in enumerate(channel):  # Goes through every messages one by one in recorded order
#             print(msg.proto)
            byteString =  msg.tensor.T[0]
            fObj = open(os.path.join(R_EXPORT_DIR, f"{idx}.jpg"), "wb")
            fObj.write(byteString)
            fObj.close()
            
#             index = idx
#             if index > 300:
#                 break
#             index = index+1
    
    if channel.name == "intention_control":
        for idx, msg in enumerate(channel):
#             print(msg.proto.x, msg.proto.y, msg.proto.z)
            labelwriter.writerow([idx, 'DLM', msg.proto.x, msg.proto.y, intentions[int(msg.proto.z)]])
            print(msg.proto)
#             index = idx
#             if index > 300:
#                 break

#     print()
                
            
cv2.destroyAllWindows()